In [1]:
!pip install -q segmentation-models-pytorch
!git clone -q https://github.com/mszczesniak02/bachlor_backup.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 7.2 MB/s eta 0:00:00


In [2]:
import os
import sys
from google.colab import drive
import glob
from pathlib import Path
import shutil
from tqdm import tqdm

In [3]:
def make_dirs():
    # make dirs for models & logs
    dirs = [
        "/content/models",

        "/content/models/segmentation",
        "/content/models/classification",
        "/content/models/autoencoder",

        "/content/models/segmentation/unet",
        "/content/models/segmentation/segformer",
        "/content/models/classification/efficienet",
        "/content/models/classification/convnext",

        "/content/models_log",

        "/content/models_log/segmentation",
        "/content/models_log/segmentation/unet",
        "/content/models_log/segmentation/segformer",

        "/content/models_log/classification",
        "/content/models_log/classification/efficienet",
        "/content/models_log/classification/convnext",
        "/content/models_log/autoencoder",
    ]
    for dir in dirs:
        os.makedirs(dir, exist_ok=True)



def mount_drive():

    drive.mount('/content/drive')
    source = '/content/drive/MyDrive/datasets/multi'  # Na Drive
    destination = '/content/datasets/multi'            # Link lokalny

    os.makedirs('/content/datasets', exist_ok=True)

    if os.path.exists(destination):
        if os.path.islink(destination):
            os.unlink(destination)
        else:
            shutil.rmtree(destination)

    # os.symlink(source, destination)
    print(f"Copying dataset from {source} to {destination}...")

    # Count files for progress bar
    total_files = sum([len(files) for r, d, files in os.walk(source)])

    with tqdm(total=total_files, desc="Copying", unit="file") as pbar:
        def copy_func(src, dst):
            shutil.copy2(src, dst)
            pbar.update(1)

        shutil.copytree(source, destination, copy_function=copy_func)

    print(f"Dataset copied!")
    print(f"  {destination} <- {source}")

def set_colab(path: str, on_colab: bool):
    dir = Path(path)
    result = list(dir.rglob("hparams.[pP][yY]"))
    for r in result:
        with open(r, "r+") as f:
            pos = f.tell()
            line = f.readline()
            if on_colab:
                if line == "ON_COLAB = False\n":
                    f.seek(pos)
                    f.write("ON_COLAB = True\n")
            else:
                if line == "ON_COLAB = True\n":
                    f.seek(pos)
                    f.write("ON_COLAB = False\n")



In [4]:
make_dirs()

In [ ]:
mount_drive()

In [6]:
set_colab("/content/bachlor_backup/src", True)

In [7]:

!cat bachlor_backup/src/segmentation/common/hparams.py | head -1
!cat bachlor_backup/src/classification/common/hparams.py | head -1
!cat bachlor_backup/src/autoencoder/hparams.py | head -1
!cat bachlor_backup/src/dim_translator/hparams.py | head -1


ON_COLAB = True
ON_COLAB = True
ON_COLAB = True
cat: bachlor_backup/src/dim_translator/hparams.py: No such file or directory


In [32]:
!rm -rd bachlor_backup/

In [8]:
!cd bachlor_backup/src/autoencoder && python train.py

2025-11-24 15:26:02.744497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763997962.763480   19485 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763997962.769085   19485 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763997962.783373   19485 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763997962.783401   19485 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763997962.783405   19485 computation_placer.cc:177] computation placer alr